In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("CShorten/ML-ArXiv-Papers")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 117592
    })
})

In [4]:
split_datasets = dataset["train"].train_test_split(test_size=0.2)

In [5]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 94073
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 23519
    })
})

In [6]:
dataset_train = split_datasets["train"]

In [7]:
dataset_train

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
    num_rows: 94073
})

In [8]:
dataset_train = dataset_train.remove_columns(['Unnamed: 0','Unnamed: 0.1'])

In [9]:
dataset_train

Dataset({
    features: ['title', 'abstract'],
    num_rows: 94073
})

In [10]:
dataset_train[0]

{'title': 'Condition monitoring and early diagnostics methodologies for hydropower\n  plants',
 'abstract': '  Hydropower plants are one of the most convenient option for power generation,\nas they generate energy exploiting a renewable source, they have relatively low\noperating and maintenance costs, and they may be used to provide ancillary\nservices, exploiting the large reservoirs of available water. The recent\nadvances in Information and Communication Technologies (ICT) and in machine\nlearning methodologies are seen as fundamental enablers to upgrade and\nmodernize the current operation of most hydropower plants, in terms of\ncondition monitoring, early diagnostics and eventually predictive maintenance.\nWhile very few works, or running technologies, have been documented so far for\nthe hydro case, in this paper we propose a novel Key Performance Indicator\n(KPI) that we have recently developed and tested on operating hydropower\nplants. In particular, we show that after more t

In [11]:
dataset_train = dataset_train.map(lambda x, idx: { 'index': idx }, with_indices=True)

Map:   0%|          | 0/94073 [00:00<?, ? examples/s]

In [12]:
dataset_train

Dataset({
    features: ['title', 'abstract', 'index'],
    num_rows: 94073
})

In [13]:
dataset_train[4]

{'title': 'MetaTune: Meta-Learning Based Cost Model for Fast and Efficient\n  Auto-tuning Frameworks',
 'abstract': '  Deep learning compiler frameworks are gaining ground as a more portable\nback-end for deep learning applications on increasingly diverse hardware.\nHowever, they face the daunting challenge of matching performance offered by\nhand-tuned target-specific libraries. While auto-tuning frameworks with\nstatistical cost models can provide dynamic and efficient code optimization,\nthey suffer from large space exploration and cost model training overheads.\nThis paper proposes MetaTune, a meta-learning based cost model that more\nquickly and accurately predicts the performance of optimized codes with\npre-trained model parameters. MetaTune encodes convolution kernel codes as\nstructurally similar graphs to facilitate meta-learning, meta-trains a GNN\nmodel with a very small input data set, and then predicts optimization\nparameters for unseen convolution operations with varyin

In [14]:
dataset_train[20572]

{'title': 'Asymptotics of $\\ell_2$ Regularized Network Embeddings',
 'abstract': '  A common approach to solving prediction tasks on large networks, such as node\nclassification or link prediction, begin by learning a Euclidean embedding of\nthe nodes of the network, from which traditional machine learning methods can\nthen be applied. This includes methods such as DeepWalk and node2vec, which\nlearn embeddings by optimizing stochastic losses formed over subsamples of the\ngraph at each iteration of stochastic gradient descent. In this paper, we study\nthe effects of adding an $\\ell_2$ penalty of the embedding vectors to the\ntraining loss of these types of methods. We prove that, under some\nexchangeability assumptions on the graph, this asymptotically leads to learning\na graphon with a nuclear-norm-type penalty, and give guarantees for the\nasymptotic distribution of the learned embedding vectors. In particular, the\nexact form of the penalty depends on the choice of subsampling m

In [15]:
import numpy as np
all_indexes = np.array(dataset_train['index'])

In [16]:
import random

def generate_hard_negatives(example, num_negatives=10,dataset=dataset_train):
    query_index = example['index'] 
    anchor = example['title']
    positive = example['abstract']
    
    negatives = []
    
    negative_indexes = np.delete(all_indexes, np.where(all_indexes == query_index))
    
    sampled_negatives = random.sample(list(negative_indexes), num_negatives)
    
    for idx in sampled_negatives:
        negatives.append(dataset[int(idx)]['abstract'])
    
    return {
        "query": anchor,
        "positive": positive,
        "negatives": negatives
    }

In [17]:
processed_data_train = dataset_train.map(generate_hard_negatives, remove_columns=dataset_train.column_names)

Map:   0%|          | 0/94073 [00:00<?, ? examples/s]

In [18]:
processed_data_train

Dataset({
    features: ['query', 'positive', 'negatives'],
    num_rows: 94073
})

In [19]:
processed_data_train[0]

{'query': 'Condition monitoring and early diagnostics methodologies for hydropower\n  plants',
 'positive': '  Hydropower plants are one of the most convenient option for power generation,\nas they generate energy exploiting a renewable source, they have relatively low\noperating and maintenance costs, and they may be used to provide ancillary\nservices, exploiting the large reservoirs of available water. The recent\nadvances in Information and Communication Technologies (ICT) and in machine\nlearning methodologies are seen as fundamental enablers to upgrade and\nmodernize the current operation of most hydropower plants, in terms of\ncondition monitoring, early diagnostics and eventually predictive maintenance.\nWhile very few works, or running technologies, have been documented so far for\nthe hydro case, in this paper we propose a novel Key Performance Indicator\n(KPI) that we have recently developed and tested on operating hydropower\nplants. In particular, we show that after more t

In [20]:
contrastive_pairs_train = []
for item in processed_data_train:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_train.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [21]:
len(contrastive_pairs_train)

94073

In [22]:
from torch.utils.data import DataLoader

In [23]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [24]:
contrastive_dataset_train = ContrastiveDataset(contrastive_pairs_train)

In [25]:
data_loader_train = DataLoader(contrastive_dataset_train, batch_size=8, shuffle=True)

In [26]:
len(data_loader_train)

3

In [27]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [28]:
model = AutoModel.from_pretrained("bert-base-uncased")

In [29]:
from peft import LoraConfig, get_peft_model

In [30]:
lora_config = LoraConfig(
    task_type= "FEATURE_EXTRACTION"
)

In [31]:
lora_model = get_peft_model(model, lora_config)

In [32]:
def cosine_distance(x, y):
    return 1 - torch.nn.functional.cosine_similarity(x, y, dim=-1)

In [33]:
def info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn):

    pos_dist = distance_fn(anchor_embedding, positive_embedding)
    neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
    
    logits = torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)
    labels = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)

    loss = torch.nn.CrossEntropyLoss()(logits, labels)
    return loss

In [34]:
import torch
import torch.optim as optim
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-5)

In [35]:
num_epochs=3

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
device

device(type='cuda')

In [38]:
torch.cuda.is_available()

True

In [39]:
lora_model = lora_model.to(device)

In [40]:
dataset_val = split_datasets["test"]

In [41]:
dataset_val

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
    num_rows: 23519
})

In [42]:
dataset_val = dataset_val.remove_columns(['Unnamed: 0','Unnamed: 0.1'])

In [43]:
dataset_val = dataset_val.map(lambda x, idx: { 'index': idx }, with_indices=True)

Map:   0%|          | 0/23519 [00:00<?, ? examples/s]

In [44]:
dataset_val

Dataset({
    features: ['title', 'abstract', 'index'],
    num_rows: 23519
})

In [45]:
all_indexes_val = np.array(dataset_val['index'])

In [46]:
def generate_hard_negatives_val(example, num_negatives=10,dataset=dataset_val):
    query_index = example['index']
    anchor = example['title']
    positive = example['abstract']
    
    negatives = []
    
    negative_indexes = np.delete(all_indexes_val, np.where(all_indexes_val == query_index))
    
    sampled_negatives = random.sample(list(negative_indexes), num_negatives)
    
    for idx in sampled_negatives:
        negatives.append(dataset[int(idx)]['abstract'])
    
    return {
        "query": anchor,
        "positive": positive,
        "negatives": negatives
    }

In [47]:
processed_data_val = dataset_val.map(generate_hard_negatives_val, remove_columns=dataset_val.column_names)

Map:   0%|          | 0/23519 [00:00<?, ? examples/s]

In [48]:
contrastive_pairs_val = []
for item in processed_data_val:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_val.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [49]:
contrastive_dataset_val = ContrastiveDataset(contrastive_pairs_val)

In [50]:
data_loader_val = DataLoader(contrastive_dataset_val, batch_size=8, shuffle=True)

In [51]:
len(data_loader_val)

3

In [52]:
def avg_embedding(inputs, model):
    input=model(**inputs)
    input_last_hidden_state=input.last_hidden_state
    input_attention_mask = inputs['attention_mask']
    input_masked_embeddings = input_last_hidden_state * input_attention_mask.unsqueeze(-1)
    input_sum_embeddings = torch.sum(input_masked_embeddings, dim=1)
    input_token_counts = torch.sum(input_attention_mask, dim=1).unsqueeze(-1)
    input_avg_embeddings = input_sum_embeddings / input_token_counts
    return(input_avg_embeddings)

In [53]:
def evaluate_mrr(model, data_loader_val, distance_fn):
    model.eval()  

    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = avg_embedding(anchor_input, model)
            positive_embedding = avg_embedding(positive_input, model)
            negative_embedding = [avg_embedding(tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device), model) for neg in negative_texts]

            pos_dist = distance_fn(anchor_embedding, positive_embedding)
            neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
            all_similarities=torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)
            
            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)

    mrr = total_rr / num_queries
    return mrr

In [54]:
import os
save_dir ="/dss/dsshome1/07/ra65bex2/srawat/average"

In [55]:
import time
epoch_metrics = []

In [56]:
torch.cuda.empty_cache()

In [57]:
for epoch in range(num_epochs):
    start_time = time.time()
    lora_model.train()
    
    total_loss = 0.0
    for batch in data_loader_train:
    
        anchor_texts = batch[0]
        positive_texts = batch[1]
        negative_texts = batch[2]
    
        anchor_inputs = tokenizer(anchor_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        positive_inputs = tokenizer(positive_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

    
        anchor_embedding = avg_embedding(anchor_inputs, lora_model)
        positive_embedding = avg_embedding(positive_inputs, lora_model)
        negative_embedding = [avg_embedding(tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device), lora_model) for neg in negative_texts]

        loss = info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn=cosine_distance)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    save_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch+1}.pth")
    torch.save(lora_model, save_path)
    print(f"EPOCH {epoch+1}:")
    print(f"Checkpoint saved: {save_path}")
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_loss / len(data_loader_train)}")
    mrr_validation = evaluate_mrr(lora_model, data_loader_val, cosine_distance)
    #mrr_train = evaluate_mrr(lora_model, data_loader_train, cosine_distance)
    #print(f"Mean Reciprocal Rank (MRR) for training set: {mrr_train:.4f}")
    print(f"Mean Reciprocal Rank (MRR) for validation set: {mrr_validation:.4f}")
    end_time = time.time()
    print(f"Epoch {epoch+1} took {(end_time - start_time) / 60:.4f} minutes.")
    print(f"\n")
    epoch_metrics.append({
        'epoch': epoch + 1,
        'training_loss': total_loss / len(data_loader_train),
        'mrr_validation': mrr_validation,
        'time_taken_minutes': (end_time - start_time) / 60
    })

EPOCH 1:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/average/checkpoint_epoch_1.pth
Epoch 1/3, Training Loss: 2.33770759900411
Mean Reciprocal Rank (MRR) for validation set: 0.2983
Epoch 1 took 0.2382 minutes.


EPOCH 2:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/average/checkpoint_epoch_2.pth
Epoch 2/3, Training Loss: 2.337019364039103
Mean Reciprocal Rank (MRR) for validation set: 0.2983
Epoch 2 took 0.1911 minutes.


EPOCH 3:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/average/checkpoint_epoch_3.pth
Epoch 3/3, Training Loss: 2.3341899712880454
Mean Reciprocal Rank (MRR) for validation set: 0.2983
Epoch 3 took 0.1808 minutes.




In [ ]:
import json

In [ ]:
with open(save_dir + '/epoch_metrics.json', 'w') as f:
    json.dump(epoch_metrics, f)